In [1]:
# import 블록
from utils import process_all_csv_files, AverageMeter, convert_secs2time, time_string
from dataloader import TrainSet
from TSRNet import TSRNet


import os
import time
import random
import math
import copy
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

In [2]:
print("train")
train_folder_path = 'C:/Users/SKTPJ15/real/traindata_au'
processed_train_data = process_all_csv_files(train_folder_path)

print("val")
val_folder_path = 'C:/Users/SKTPJ15/real/valdata'
processed_val_data = process_all_csv_files(val_folder_path)

train
Processing file: C:/Users/SKTPJ15/real/traindata_au\augmented_cluster0_nk100_2642_2024-04-14_12-14-19.dat.csv
Processing file: C:/Users/SKTPJ15/real/traindata_au\augmented_cluster0_nk100_2642_2024-04-14_12-24-19.dat.csv
Processing file: C:/Users/SKTPJ15/real/traindata_au\augmented_cluster0_nk100_2642_2024-04-14_12-34-19.dat.csv
Processing file: C:/Users/SKTPJ15/real/traindata_au\augmented_cluster0_nk100_2642_2024-04-14_12-44-19.dat.csv
Processing file: C:/Users/SKTPJ15/real/traindata_au\augmented_cluster0_nk100_2642_2024-04-14_12-54-19.dat.csv
Processing file: C:/Users/SKTPJ15/real/traindata_au\augmented_cluster0_nk100_2642_2024-04-14_13-04-19.dat.csv
Processing file: C:/Users/SKTPJ15/real/traindata_au\augmented_cluster0_nk100_2642_2024-04-14_13-14-19.dat.csv
Processing file: C:/Users/SKTPJ15/real/traindata_au\augmented_cluster0_nk100_2642_2024-04-14_13-24-19.dat.csv
Processing file: C:/Users/SKTPJ15/real/traindata_au\augmented_cluster0_nk100_2642_2024-04-14_13-34-19.dat.csv
Proc

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}

train_dataset = TrainSet(processed_train_data)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, **kwargs)

val_dataset = TrainSet(processed_val_data)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, **kwargs)

In [4]:
# 시드 값 고정
random.seed(831)
torch.manual_seed(831)
torch.cuda.manual_seed_all(831)

# 모델, 하이퍼 파라미터 설정
model = TSRNet(enc_in=3).to(device)
optimizer = torch.optim.AdamW(model.parameters() , lr=1e-4, weight_decay=1e-4)
lr = 1e-4
epochs = 64

n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {round(n_parameters * 1e-6, 2)} M")

Total trainable parameters: 4.65 M


In [5]:
def train(model, epoch, train_loader, optimizer):
    mask_ratio_time = 30
    
    model.train()
    
    total_losses = AverageMeter()
    reconstruction_errors = AverageMeter()
    
    for i, (time_bcg, spectrogram_bcg) in tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        
        # time branch masking
        time_bcg = time_bcg.float().to(device)  #(32, 560, 3)
        bs, time_length, dim = time_bcg.shape
        mask_time = copy.deepcopy(time_bcg)
        mask = torch.zeros((bs,time_length,1), dtype=torch.bool).to(device)
        patch_length = time_length // 100 #5
        for j in random.sample(range(0,100), mask_ratio_time):
            mask[:, j*patch_length:(j+1)*patch_length] = 1 #(32, 5, 1)
        mask_time = torch.mul(mask_time, ~mask)
        
        
        #spectrogram branch
        spec_bcg = spectrogram_bcg.float().to(device) #(32, 63, 66, 12)
        (gen_time, time_var) = model(mask_time, spec_bcg)
        
        #loss time bcg
        time_err = (gen_time - time_bcg) ** 2
        reconstruction_error = torch.mean(time_err).item()
        l_time = torch.mean(torch.exp(-time_var)*time_err) + torch.mean(time_var) 
        loss = l_time 

        # back propagation
        loss.backward()
        optimizer.step()

        total_losses.update(loss.item(), bs)
        reconstruction_errors.update(reconstruction_error, bs)
        
    print(f'Train Epoch: {epoch} Total_Loss: {total_losses.avg:.6f} Reconstruction_Error: {reconstruction_errors.avg:.6f}')
    
    return {
        "epoch": epoch,
        "train_total_loss": total_losses.avg,
        "train_reconstruction_error": reconstruction_errors.avg
    }
        
def val(model, test_loader):
    torch.zero_grad = True
    
    model.eval()
    
    total_losses = AverageMeter()
    reconstruction_errors = AverageMeter()
    
    for i, (time_bcg, spectrogram_bcg) in tqdm(enumerate(test_loader)):
        with torch.no_grad():
            time_bcg = time_bcg.float().to(device)
            spec_bcg = spectrogram_bcg.float().to(device)
            
            # Forward pass
            (gen_time, time_var) = model(time_bcg, spec_bcg)
            
            # Evaluate model
            time_err = (gen_time - time_bcg) ** 2
            reconstruction_error = torch.mean(time_err).item()
            l_time = torch.mean(torch.exp(-time_var) * time_err) + torch.mean(time_var)
            loss = l_time

            total_losses.update(loss.item(), time_bcg.size(0))
            reconstruction_errors.update(reconstruction_error, time_bcg.size(0))
    
    print(f'Test Total_Loss: {total_losses.avg:.6f} Reconstruction_Error: {reconstruction_errors.avg:.6f}')
    
    return {
        "val_total_loss": total_losses.avg,
        "val_reconstruction_error": reconstruction_errors.avg
    }
    
def adjust_learning_rate(optimizer, init_lr, epoch, epochs):
    """Decay the learning rate based on schedule"""
    cur_lr = init_lr * 0.5 * (1. + math.cos(math.pi * epoch / epochs))
    for param_group in optimizer.param_groups:
        param_group['lr'] = cur_lr
        

In [6]:
pth_path = None
save_model = True
save_path = "new_save0/"

# continue training
if pth_path is not None:
    checkpoint = torch.load(pth_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Results 저장용 리스트
results = []

# start training
start_time = time.time()
epoch_time = AverageMeter()

for epoch in range(0,epochs):
    adjust_learning_rate(optimizer, lr, epoch, epochs)
    need_hour, need_mins, need_secs = convert_secs2time(epoch_time.avg * (epochs - epoch))
    need_time = '[Need: {:02d}:{:02d}:{:02d}]'.format(need_hour, need_mins, need_secs)
    print(' {:3d}/{:3d} ----- [{:s}] {:s}'.format(epoch, epochs, time_string(), need_time))
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

    train_metrics = train(model, epoch, train_loader, optimizer)
    test_metrics = val(model, val_loader) 
    
    combined_metrics = {**train_metrics, **test_metrics}
    results.append(combined_metrics)
    
    if save_model == 1:
        torch.save({'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict()}, save_path + 'TSRNet-%d.pt' % epoch)
        
# 결과를 CSV 파일로 저장
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(save_path, 'metrics.csv'), index=False)


   0/ 64 ----- [[2024-08-28 14:35:27]] [Need: 00:00:00]


338it [00:14, 23.20it/s]


Train Epoch: 0 Total_Loss: 0.324174 Reconstruction_Error: 0.089938


11356it [00:47, 237.80it/s]


Test Total_Loss: 0.242482 Reconstruction_Error: 0.069374
   1/ 64 ----- [[2024-08-28 14:36:53]] [Need: 00:00:00]


...

Test Total_Loss: 0.015466 Reconstruction_Error: 0.015458
  63/ 64 ----- [[2024-08-28 16:06:32]] [Need: 00:01:25]


338it [00:11, 30.13it/s]


Train Epoch: 63 Total_Loss: 0.011742 Reconstruction_Error: 0.011734


11356it [00:43, 262.46it/s]

Test Total_Loss: 0.015461 Reconstruction_Error: 0.015453
